In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [2]:
movies = pd.read_csv("ml-1m/movies.dat", delimiter="::", engine="python", names=["MovieID", "Title", "Genres"])
users = pd.read_csv("ml-1m/users.dat", delimiter="::", engine="python", names=["UserID", "Gender", "Age", "Occupation", "ZipCode"])
ratings = pd.read_csv("ml-1m/ratings.dat", delimiter="::", engine="python", names=["UserID", "MovieID", "Rating", "Timestamp"])

movies["Year"] = movies.Title.str[-5:-1]
movies["Year"] = movies["Year"].astype(int)
movies.Title = movies.Title.str[:-7]

In [3]:
db_name = 'movies.db'
con = sqlite3.connect(db_name)

movies.to_sql("movies", con, if_exists='replace', index=False)
users.to_sql("users", con, if_exists='replace', index=False)
ratings.to_sql("ratings", con, if_exists='replace', index=False)

In [4]:
sql_result = pd.read_sql_query("SELECT * FROM movies", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
3878,3948,Meet the Parents,Comedy,2000
3879,3949,Requiem for a Dream,Drama,2000
3880,3950,Tigerland,Drama,2000
3881,3951,Two Family House,Drama,2000


In [15]:
sql_result = pd.read_sql_query("""
SELECT Title
FROM movies
where title NOT like "T%"
union
select * from (
SELECT Title
FROM movies
where title like "T%"
ORDER BY TITLE)
""", con)
sql_result

,Title
0,"$1,000,000 Duck"
1,'Night Mother
2,'Til There Was You
3,"'burbs, The"
4,...And Justice for All
...,...
3836,Zero Effect
3837,Zero Kelvin (Kj�rlighetens kj�tere)
3838,Zeus and Roxanne
3839,Zone 39


In [9]:
sql_result = pd.read_sql_query("""
select * from (
SELECT Title
FROM movies
where title like "T%"
ORDER BY TITLE)
""", con)
sql_result

,Title
0,T-Men
1,Taffin
2,Tainted
3,Take the Money and Run
4,"Taking of Pelham One Two Three, The"
...,...
228,Two Much
229,Two Thousand Maniacs!
230,Two Women (La Ciociara)
231,Two if by Sea


# 1. Wybierz wszystkie kolumny (z users)

In [13]:
sql_result = pd.read_sql_query("SELECT * FROM users", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


# 2. Wybierz płeć i wiek użytkowników

In [14]:
sql_result = pd.read_sql_query("SELECT Gender, Age FROM users", con)
sql_result

,Gender,Age
0,F,1
1,M,56
2,M,25
3,M,45
4,M,25
...,...,...
6035,F,25
6036,F,45
6037,F,56
6038,F,45


# 3. Wybierz pierwszych 5 filmów.

In [8]:
sql_result = pd.read_sql_query("SELECT * FROM movies LIMIT 5", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995


In [9]:
sql_result = pd.read_sql_query("SELECT Title FROM movies LIMIT 5", con)
sql_result

,Title
0,Toy Story
1,Jumanji
2,Grumpier Old Men
3,Waiting to Exhale
4,Father of the Bride Part II


# 4. Wybierz filmy, których ID to nie jest ani 3, ani 5.

* `<>`, `!=`, `AND`
* `IN`, `NOT`

In [11]:
sql_result = pd.read_sql_query("SELECT * FROM movies WHERE MovieID <> 3 AND MovieID != 5", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,4,Waiting to Exhale,Comedy|Drama,1995
3,6,Heat,Action|Crime|Thriller,1995
4,7,Sabrina,Comedy|Romance,1995
...,...,...,...,...
3876,3948,Meet the Parents,Comedy,2000
3877,3949,Requiem for a Dream,Drama,2000
3878,3950,Tigerland,Drama,2000
3879,3951,Two Family House,Drama,2000


In [12]:
# polecam
sql_result = pd.read_sql_query("SELECT * FROM movies WHERE MovieID NOT IN (3, 5)", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,4,Waiting to Exhale,Comedy|Drama,1995
3,6,Heat,Action|Crime|Thriller,1995
4,7,Sabrina,Comedy|Romance,1995
...,...,...,...,...
3876,3948,Meet the Parents,Comedy,2000
3877,3949,Requiem for a Dream,Drama,2000
3878,3950,Tigerland,Drama,2000
3879,3951,Two Family House,Drama,2000


In [13]:
# nie polecam, ale w ramach ciekawostki
sql_result = pd.read_sql_query("SELECT * FROM movies WHERE NOT MovieID IN (3, 5)", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,4,Waiting to Exhale,Comedy|Drama,1995
3,6,Heat,Action|Crime|Thriller,1995
4,7,Sabrina,Comedy|Romance,1995
...,...,...,...,...
3876,3948,Meet the Parents,Comedy,2000
3877,3949,Requiem for a Dream,Drama,2000
3878,3950,Tigerland,Drama,2000
3879,3951,Two Family House,Drama,2000


# 5. Wybierz użytkowników, których wiek jest z przedziału [18, 35].

* `AND`, `>=`, `<=`
* `BETWEEN`

In [14]:
sql_result = pd.read_sql_query("SELECT * FROM users WHERE Age >= 18 AND Age <= 35", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,3,M,25,15,55117
1,5,M,25,20,55455
2,7,M,35,1,06810
3,8,M,25,12,11413
4,9,M,25,17,61614
...,...,...,...,...,...
4387,6031,F,18,0,45123
4388,6034,M,25,14,94117
4389,6035,F,25,1,78734
4390,6036,F,25,15,32603


In [15]:
sql_result = pd.read_sql_query("SELECT * FROM users WHERE Age BETWEEN 18 AND 35", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,3,M,25,15,55117
1,5,M,25,20,55455
2,7,M,35,1,06810
3,8,M,25,12,11413
4,9,M,25,17,61614
...,...,...,...,...,...
4387,6031,F,18,0,45123
4388,6034,M,25,14,94117
4389,6035,F,25,1,78734
4390,6036,F,25,15,32603


# 6. Wybierz użytkowników, których wiek jest z przedziału < 18 lub > 35.

In [16]:
sql_result = pd.read_sql_query("SELECT * FROM users WHERE Age < 18 OR Age > 35", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,4,M,45,7,02460
3,6,F,50,9,55117
4,13,M,45,1,93304
...,...,...,...,...,...
1643,6032,M,45,7,55108
1644,6033,M,50,13,78232
1645,6037,F,45,1,76006
1646,6038,F,56,1,14706


In [17]:
# od biedy może być, ale może zmniejszać czytelność
sql_result = pd.read_sql_query("SELECT * FROM users WHERE NOT (Age >= 18 AND Age <= 35)", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,4,M,45,7,02460
3,6,F,50,9,55117
4,13,M,45,1,93304
...,...,...,...,...,...
1643,6032,M,45,7,55108
1644,6033,M,50,13,78232
1645,6037,F,45,1,76006
1646,6038,F,56,1,14706


In [15]:
# polecam
sql_result = pd.read_sql_query("SELECT * FROM users WHERE Age NOT BETWEEN 18 AND 35", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,4,M,45,7,02460
3,6,F,50,9,55117
4,13,M,45,1,93304
...,...,...,...,...,...
1643,6032,M,45,7,55108
1644,6033,M,50,13,78232
1645,6037,F,45,1,76006
1646,6038,F,56,1,14706


In [16]:
# raczej tak nie piszemy
sql_result = pd.read_sql_query("SELECT * FROM users WHERE NOT Age BETWEEN 18 AND 35", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,4,M,45,7,02460
3,6,F,50,9,55117
4,13,M,45,1,93304
...,...,...,...,...,...
1643,6032,M,45,7,55108
1644,6033,M,50,13,78232
1645,6037,F,45,1,76006
1646,6038,F,56,1,14706


# 7. Wybierz filmy, których rok produkcji jest parzysty, a tytuł filmu zaczyna się na literę T.

In [23]:
sql_result = pd.read_sql_query("""
SELECT * FROM movies WHERE year % 2 = 0 AND Title LIKE 'T%'
""", con)
sql_result

,MovieID,Title,Genres,Year
0,64,Two if by Sea,Comedy|Romance,1996
1,111,Taxi Driver,Drama|Thriller,1976
2,306,Three Colors: Red,Drama,1994
3,308,Three Colors: White,Drama,1994
4,326,To Live (Huozhe),Drama,1994
...,...,...,...,...
111,3852,"Tao of Steve, The",Comedy,2000
112,3853,"Tic Code, The",Drama,1998
113,3891,Turn It Up,Crime|Drama,2000
114,3950,Tigerland,Drama,2000


In [22]:
sql_result = pd.read_sql_query("""
SELECT * FROM movies WHERE year % 2 = 0 AND Title LIKE 'Ham%'
""", con)
sql_result

,MovieID,Title,Genres,Year
0,1411,Hamlet,Drama,1996
1,1941,Hamlet,Drama,1948
2,2820,Hamlet,Drama,1964
3,3598,Hamlet,Drama,2000
4,3723,Hamlet,Drama,1990


In [24]:
sql_result = pd.read_sql_query("""
SELECT DISTINCT Title FROM movies WHERE year % 2 = 0 AND Title LIKE 'Ham%'
""", con)
sql_result

,Title
0,Hamlet


# 8. Wybierz filmy, których rok produkcji jest parzysty lub jest mniejszy od 1995.

In [26]:
sql_result = pd.read_sql_query("""
SELECT * FROM movies WHERE year % 2 = 0 OR year <= 1995
""", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
3280,3948,Meet the Parents,Comedy,2000
3281,3949,Requiem for a Dream,Drama,2000
3282,3950,Tigerland,Drama,2000
3283,3951,Two Family House,Drama,2000


# 9. Wybierz unikatowe grupy wiekowe użytkowników.

In [27]:
sql_result = pd.read_sql_query("""
SELECT DISTINCT Age FROM users
""", con)
sql_result

,Age
0,1
1,56
2,25
3,45
4,50
5,35
6,18


# 10. Wybierz unikatowe kombinacje wieku i płci użytkowników.

In [28]:
sql_result = pd.read_sql_query("""
SELECT DISTINCT Age, Gender FROM users
""", con)
sql_result

,Age,Gender
0,1,F
1,56,M
2,25,M
3,45,M
4,50,F
5,35,M
6,35,F
7,25,F
8,50,M
9,18,F


# 11. Wybierz zawód użytkownika, gdzie wiek zawiera się w [18, 35], ale ogranicz się do pierwszych 20 wierszy.

In [31]:
# wybieramy kolumnę Age aby się upewnić
sql_result = pd.read_sql_query("""
SELECT Occupation, Age FROM users WHERE Age BETWEEN 18 AND 35 LIMIT 20
""", con)
sql_result

,Occupation,Age
0,15,25
1,20,25
2,1,35
3,12,25
4,17,25
5,1,35
6,1,25
7,12,25
8,0,35
9,7,25


# 12. Wybierz filmy, mieszanka gatunków to Comedy|Romance, Action|Crime|Thriller lub Comedy|Horror

In [32]:
sql_result = pd.read_sql_query("""
SELECT * 
FROM movies 
WHERE Genres IN ('Comedy|Romance', 'Action|Crime|Thriller', 'Comedy|Horror')
""", con)
sql_result

,MovieID,Title,Genres,Year
0,3,Grumpier Old Men,Comedy|Romance,1995
1,6,Heat,Action|Crime|Thriller,1995
2,7,Sabrina,Comedy|Romance,1995
3,12,Dracula: Dead and Loving It,Comedy|Horror,1995
4,39,Clueless,Comedy|Romance,1995
...,...,...,...,...
174,3829,Mad About Mambo,Comedy|Romance,2000
175,3874,"Couch in New York, A",Comedy|Romance,1996
176,3885,Love & Sex,Comedy|Romance,2000
177,3909,Woman on Top,Comedy|Romance,2000


# 13. Wybierz filmy, mieszanka gatunków to Comedy|Romance, Action|Crime|Thriller lub Comedy|Horror, a rok produkcji jest większy niż 1985.

In [33]:
sql_result = pd.read_sql_query("""
SELECT * 
FROM movies 
WHERE Genres IN ('Comedy|Romance', 'Action|Crime|Thriller', 'Comedy|Horror')
AND Year >= 1985
""", con)
sql_result

,MovieID,Title,Genres,Year
0,3,Grumpier Old Men,Comedy|Romance,1995
1,6,Heat,Action|Crime|Thriller,1995
2,7,Sabrina,Comedy|Romance,1995
3,12,Dracula: Dead and Loving It,Comedy|Horror,1995
4,39,Clueless,Comedy|Romance,1995
...,...,...,...,...
147,3785,Scary Movie,Comedy|Horror,2000
148,3829,Mad About Mambo,Comedy|Romance,2000
149,3874,"Couch in New York, A",Comedy|Romance,1996
150,3885,Love & Sex,Comedy|Romance,2000


# Sortowanie

# 1. Posortuj filmy po tytule.

In [34]:
sql_result = pd.read_sql_query("""
SELECT * 
FROM movies 
ORDER BY Title
""", con)
sql_result

,MovieID,Title,Genres,Year
0,2031,"$1,000,000 Duck",Children's|Comedy,1971
1,3112,'Night Mother,Drama,1986
2,779,'Til There Was You,Drama|Romance,1997
3,2072,"'burbs, The",Comedy,1989
4,3420,...And Justice for All,Drama|Thriller,1979
...,...,...,...,...
3878,1845,Zero Effect,Comedy|Thriller,1998
3879,1364,Zero Kelvin (Kj�rlighetens kj�tere),Action,1995
3880,1426,Zeus and Roxanne,Children's,1997
3881,2698,Zone 39,Sci-Fi,1997


# 2. Posortuj filmy po tytule, ale tylko te, których tytuł zaczyna się na literę T

In [35]:
sql_result = pd.read_sql_query("""
SELECT * 
FROM movies 
WHERE Title LIKE 'T%'
ORDER BY Title
""", con)
sql_result

,MovieID,Title,Genres,Year
0,1154,T-Men,Film-Noir,1947
1,3523,Taffin,Action|Thriller,1988
2,1764,Tainted,Comedy|Thriller,1998
3,1963,Take the Money and Run,Comedy,1969
4,3384,"Taking of Pelham One Two Three, The",Action,1974
...,...,...,...,...
228,618,Two Much,Comedy|Romance,1996
229,3351,Two Thousand Maniacs!,Horror,1964
230,3808,Two Women (La Ciociara),Drama|War,1961
231,64,Two if by Sea,Comedy|Romance,1996


# 3. Wybierz unikatowe grupy wiekowe użytkowników i posortuj.

In [36]:
sql_result = pd.read_sql_query("""
SELECT DISTINCT Age
FROM users 
ORDER BY Age
""", con)
sql_result

,Age
0,1
1,18
2,25
3,35
4,45
5,50
6,56


# 4. Posortuj filmy po roku produkcji malejąco, a w ramach jednego roku po tytule rosnąco.

In [37]:
sql_result = pd.read_sql_query("""
SELECT *
FROM movies 
ORDER BY Year DESC, Title
""", con)
sql_result

,MovieID,Title,Genres,Year
0,3534,28 Days,Comedy,2000
1,3322,3 Strikes,Comedy,2000
2,3862,About Adam,Comedy,2000
3,3754,"Adventures of Rocky and Bullwinkle, The",Animation|Children's|Comedy,2000
4,3897,Almost Famous,Comedy|Drama,2000
...,...,...,...,...
3878,3309,"Dog's Life, A",Comedy,1920
3879,3231,"Saphead, The",Comedy,1920
3880,3132,Daddy Long Legs,Comedy,1919
3881,2821,Male and Female,Adventure|Drama,1919


# 5. Wybierz unikatowe kombinacje wieku i płci użytkowników i posortuj, najpierw po płci, a potem po wieku.

In [40]:
sql_result = pd.read_sql_query("""
SELECT DISTINCT Gender, Age
FROM users 
ORDER BY Gender, Age
""", con)
sql_result

,Gender,Age
0,F,1
1,F,18
2,F,25
3,F,35
4,F,45
5,F,50
6,F,56
7,M,1
8,M,18
9,M,25


# 6. Wybierz unikatowe kombinacje wieku i płci użytkowników, wybierz tylko pełnoletnie kobiety i posortuj malejąco po wieku.

In [41]:
sql_result = pd.read_sql_query("""
SELECT DISTINCT Gender, Age
FROM users 
WHERE Gender = 'F' AND Age >= 18
ORDER BY Age DESC
""", con)
sql_result

,Gender,Age
0,F,56
1,F,50
2,F,45
3,F,35
4,F,25
5,F,18


# Grupowanie

# 1. Wybierz liczbę filmów o danej mieszaninie gatunków.

przykładowa mieszanina: `Comedy|Romance`

In [42]:
sql_result = pd.read_sql_query("""
SELECT Genres, COUNT(*)
FROM movies 
GROUP BY Genres
""", con)
sql_result

,Genres,COUNT(*)
0,Action,65
1,Action|Adventure,25
2,Action|Adventure|Animation,1
3,Action|Adventure|Animation|Children's|Fantasy,1
4,Action|Adventure|Animation|Horror|Sci-Fi,1
...,...,...
296,Sci-Fi|Thriller|War,1
297,Sci-Fi|War,1
298,Thriller,101
299,War,12


# 2. Wybierz unikatowe kombinacje wieku i płci użytkowników, zlicz, ile jest rekordów dla każdej kombinacji i posortuj, najpierw po płci, a potem po wieku(a osobno po liczbie reprezentantów każdej grupy).

In [44]:
sql_result = pd.read_sql_query("""
SELECT Age, Gender, COUNT(*)
FROM users 
GROUP BY 1, 2
ORDER BY 2, 1
""", con)
sql_result

,Age,Gender,COUNT(*)
0,1,F,78
1,18,F,298
2,25,F,558
3,35,F,338
4,45,F,189
5,50,F,146
6,56,F,102
7,1,M,144
8,18,M,805
9,25,M,1538


In [46]:
sql_result = pd.read_sql_query("""
SELECT Age, Gender, COUNT(*)
FROM users 
GROUP BY 1, 2
ORDER BY 3 DESC
""", con)
sql_result

,Age,Gender,COUNT(*)
0,25,M,1538
1,35,M,855
2,18,M,805
3,25,F,558
4,45,M,361
5,50,M,350
6,35,F,338
7,18,F,298
8,56,M,278
9,45,F,189


# 3. Wybierz minimalną, maksymalną i średnią ocenę dla każdego filmu (bez join).

In [48]:
sql_result = pd.read_sql_query("""
SELECT *
FROM ratings 
LIMIT 50
""", con)
sql_result

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [49]:
sql_result = pd.read_sql_query("""
SELECT MovieID, MIN(Rating), AVG(Rating), MAX(Rating)
FROM ratings 
GROUP BY MovieID
""", con)
sql_result

,MovieID,MIN(Rating),AVG(Rating),MAX(Rating)
0,1,1,4.146846,5
1,2,1,3.201141,5
2,3,1,3.016736,5
3,4,1,2.729412,5
4,5,1,3.006757,5
...,...,...,...,...
3701,3948,1,3.635731,5
3702,3949,1,4.115132,5
3703,3950,1,3.666667,5
3704,3951,1,3.900000,5


In [50]:
# modyfikacja
sql_result = pd.read_sql_query("""
SELECT Rating, COUNT(*)
FROM ratings 
GROUP BY Rating
""", con)
sql_result

,Rating,COUNT(*)
0,1,56174
1,2,107557
2,3,261197
3,4,348971
4,5,226310


# 4. Ile filmów z literą X w tytule pojawiało się w poszczególnych latach?

In [51]:
sql_result = pd.read_sql_query("""
SELECT Year, COUNT(*)
FROM movies
WHERE Title LIKE '%X%'
GROUP BY Year
""", con)
sql_result

,Year,COUNT(*)
0,1929,1
1,1934,1
2,1947,1
3,1950,1
4,1956,1
5,1958,1
6,1959,1
7,1971,1
8,1972,2
9,1973,1


# 5. Wybierz lata i liczbę filmów, których tytuły zawierają literę X, ale tylko takie lata, że takich filmów było więcej niż 3.

In [53]:
sql_result = pd.read_sql_query("""
SELECT Year, COUNT(*)
FROM movies
WHERE Title LIKE '%X%'
GROUP BY Year
HAVING COUNT(*) > 3
""", con)
sql_result

,Year,COUNT(*)
0,1981,4
1,1982,4
2,1986,5
3,1989,4
4,1990,4
5,1993,6
6,1994,10
7,1995,6
8,1996,5
9,1997,6


# 6. Wybierz lata i liczbę filmów i posortuj od najpłodniejszego roku do najmniej, zostaw jedynie pierwszych 10 wierszy.

In [54]:
sql_result = pd.read_sql_query("""
SELECT Year, COUNT(*)
FROM movies
GROUP BY Year
ORDER BY COUNT(*) DESC
LIMIT 10
""", con)
sql_result

,Year,COUNT(*)
0,1996,345
1,1995,342
2,1998,337
3,1997,315
4,1999,283
5,1994,257
6,1993,165
7,2000,156
8,1986,104
9,1992,102


# Join

# 1. Połącz ratings z movies

In [55]:
# domyślnie jest inner join
# ale możemy zamiast JOIN napisać np. LEFT JOIN
sql_result = pd.read_sql_query("""
SELECT *
FROM ratings as r
JOIN movies as m
ON r.MovieID = m.MovieID
""", con)
sql_result

,UserID,MovieID,Rating,Timestamp,MovieID,Title,Genres,Year
0,1,1193,5,978300760,1193,One Flew Over the Cuckoo's Nest,Drama,1975
1,1,661,3,978302109,661,James and the Giant Peach,Animation|Children's|Musical,1996
2,1,914,3,978301968,914,My Fair Lady,Musical|Romance,1964
3,1,3408,4,978300275,3408,Erin Brockovich,Drama,2000
4,1,2355,5,978824291,2355,"Bug's Life, A",Animation|Children's|Comedy,1998
...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,1091,Weekend at Bernie's,Comedy,1989
1000205,6040,1094,5,956704887,1094,"Crying Game, The",Drama|Romance|War,1992
1000206,6040,562,5,956704746,562,Welcome to the Dollhouse,Comedy|Drama,1995
1000207,6040,1096,4,956715648,1096,Sophie's Choice,Drama,1982


In [56]:
sql_result = pd.read_sql_query("""
SELECT r.MovieID, Rating, Title
FROM ratings as r
JOIN movies as m
ON r.MovieID = m.MovieID
""", con)
sql_result

,MovieID,Rating,Title
0,1193,5,One Flew Over the Cuckoo's Nest
1,661,3,James and the Giant Peach
2,914,3,My Fair Lady
3,3408,4,Erin Brockovich
4,2355,5,"Bug's Life, A"
...,...,...,...
1000204,1091,1,Weekend at Bernie's
1000205,1094,5,"Crying Game, The"
1000206,562,5,Welcome to the Dollhouse
1000207,1096,4,Sophie's Choice


# 2. Jaki jest najlepszy film wszechczasów?

* albo join i potem groupby
* albo zrobić podzapytanie które robi groupby i dopiero potem join

In [28]:
# join i potem groupby

# tutaj łamiemy zasadę, aby nie wstawiać kolumny po której nie grupujemy
# robimy to świadomie, bo MovieID wyznacza jednoznacznie tytuł
# (warto zauważyć, że w drugą stronę tak nie jest, np. tytuł Hamlet
# oznacza wiele filmów o różnych MovieID)

sql_result = pd.read_sql_query("""
SELECT r.MovieID, Title, AVG(Rating)
FROM ratings as r
JOIN movies as m
ON r.MovieID = m.MovieID
GROUP BY r.MovieID
ORDER BY AVG(Rating) DESC
""", con)
sql_result

,MovieID,Title,AVG(Rating)
0,3881,Bittersweet Motel,5.0
1,3656,Lured,5.0
2,3607,One Little Indian,5.0
3,3382,Song of Freedom,5.0
4,3280,"Baby, The",5.0
...,...,...,...
3701,655,Mutters Courage,1.0
3702,641,"Little Indian, Big City (Un indien dans la ville)",1.0
3703,142,Shadows (Cienie),1.0
3704,133,Nueba Yol,1.0


In [71]:
# podzapytanie

sql_result = pd.read_sql_query("""
SELECT m.MovieID, Title, ocena
FROM movies as m
JOIN (SELECT MovieID, AVG(Rating) as ocena
FROM ratings 
GROUP BY MovieID) o
ON m.MovieID = o.MovieID
ORDER BY ocena DESC
""", con)
sql_result

,MovieID,Title,ocena
0,787,"Gate of Heavenly Peace, The",5.0
1,989,Schlafes Bruder (Brother of Sleep),5.0
2,1830,Follow the Bitch,5.0
3,3172,Ulysses (Ulisse),5.0
4,3233,Smashing Time,5.0
...,...,...,...
3701,3376,"Fantastic Night, The (La Nuit Fantastique)",1.0
3702,3460,Hillbillys in a Haunted House,1.0
3703,3493,Torso (Corpi Presentano Tracce di Violenza Car...,1.0
3704,3651,"Blood Spattered Bride, The (La Novia Ensangren...",1.0


In [72]:
%%timeit

sql_result = pd.read_sql_query("""
SELECT r.MovieID, Title, AVG(Rating)
FROM ratings as r
JOIN movies as m
ON r.MovieID = m.MovieID
GROUP BY r.MovieID
ORDER BY AVG(Rating) DESC
""", con)
sql_result

2.25 s ± 56.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
%%timeit

sql_result = pd.read_sql_query("""
SELECT m.MovieID, Title, ocena
FROM movies as m
JOIN (SELECT MovieID, AVG(Rating) as ocena
FROM ratings 
GROUP BY MovieID) o
ON m.MovieID = o.MovieID
ORDER BY ocena DESC
""", con)
sql_result

1.11 s ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 3. Bierzemy pod uwagę filmy ze >= 100 głosami

In [76]:
sql_result = pd.read_sql_query("""
SELECT r.MovieID, Title, AVG(Rating), COUNT(*)
FROM ratings as r
JOIN movies as m
ON r.MovieID = m.MovieID
GROUP BY r.MovieID
HAVING COUNT(*) >= 100
ORDER BY AVG(Rating) DESC
""", con)
sql_result

,MovieID,Title,AVG(Rating),COUNT(*)
0,2019,Seven Samurai (The Magnificent Seven) (Shichin...,4.560510,628
1,318,"Shawshank Redemption, The",4.554558,2227
2,858,"Godfather, The",4.524966,2223
3,745,"Close Shave, A",4.520548,657
4,50,"Usual Suspects, The",4.517106,1783
...,...,...,...,...
2014,2383,Police Academy 6: City Under Siege,1.657718,149
2015,2817,Aces: Iron Eagle III,1.640000,125
2016,3799,Pok�mon the Movie 2000,1.620000,100
2017,3593,Battlefield Earth,1.611111,342


In [77]:
sql_result = pd.read_sql_query("""
SELECT m.MovieID, Title, ocena
FROM movies as m
JOIN (SELECT MovieID, AVG(Rating) as ocena, COUNT(*) as licznosc
FROM ratings 
GROUP BY MovieID
HAVING licznosc >= 100) o
ON m.MovieID = o.MovieID
ORDER BY ocena DESC
""", con)
sql_result

,MovieID,Title,ocena
0,2019,Seven Samurai (The Magnificent Seven) (Shichin...,4.560510
1,318,"Shawshank Redemption, The",4.554558
2,858,"Godfather, The",4.524966
3,745,"Close Shave, A",4.520548
4,50,"Usual Suspects, The",4.517106
...,...,...,...
2014,2383,Police Academy 6: City Under Siege,1.657718
2015,2817,Aces: Iron Eagle III,1.640000
2016,3799,Pok�mon the Movie 2000,1.620000
2017,3593,Battlefield Earth,1.611111


# 4. Wyświetl pełną nazwę filmu, który miał najwięcej głosów

In [79]:
sql_result = pd.read_sql_query("""
SELECT r.MovieID, Title, COUNT(*)
FROM ratings as r
JOIN movies as m
ON r.MovieID = m.MovieID
GROUP BY r.MovieID
ORDER BY COUNT(*) DESC
LIMIT 1
""", con)
sql_result

,MovieID,Title,COUNT(*)
0,2858,American Beauty,3428


In [81]:
sql_result = pd.read_sql_query("""
SELECT m.MovieID, Title, licznosc
FROM movies as m
JOIN (SELECT MovieID, COUNT(*) as licznosc
FROM ratings 
GROUP BY MovieID) o
ON m.MovieID = o.MovieID
ORDER BY licznosc DESC
LIMIT 1
""", con)
sql_result

,MovieID,Title,licznosc
0,2858,American Beauty,3428


### cte (common table expression)



Bez cte można zrobić tak

In [26]:
sql_result = pd.read_sql_query("""
select a.* from
(
select * from users
) a limit 5
""", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


A z CTE można tak:

In [27]:
sql_result = pd.read_sql_query("""
with tab1 as(
select * from users
)
select * from tab1 limit 5
""", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
